In [1]:
import numpy as np 
import pandas as pd
import pickle as pkl
import csv
import os
import torch
from torch_geometric.data import Data

In [3]:
np.random.seed(42)

In [14]:
df=pd.read_csv(open('F:\CODES\codePractices\pytorch\pytorch_geometric_YooChoose\data\input\yoochoose-data\yoochoose-clicks.dat',
               'r'),nrows=20000,names=['session_id','timestamp','item_id','category'],header=None,engine='python',sep=',')
df.head(5)
#df.columns=['session_id','timestamp','item_id','category']


,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,214536502,0
1,1,2014-04-07T10:54:09.868Z,214536500,0
2,1,2014-04-07T10:54:46.998Z,214536506,0
3,1,2014-04-07T10:57:00.306Z,214577561,0
4,2,2014-04-07T13:56:37.614Z,214662742,0


In [15]:
buy_df=pd.read_csv(open('F:\CODES\codePractices\pytorch\pytorch_geometric_YooChoose\data\input\yoochoose-data\yoochoose-buys.dat',
               'r'),nrows=20000,names=['session_id','timestamp','item_id','price','quantity'])
buy_df.head(5)

,session_id,timestamp,item_id,price,quantity
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1


In [16]:
buy_df.nunique()

session_id    10405
timestamp     19828
item_id        3402
price           305
quantity         18
dtype: int64

In [17]:
df.nunique()

session_id     4968
timestamp     19999
item_id        4425
category          1
dtype: int64

In [50]:
df['valid_session']=df.session_id.map(df.groupby('session_id')['item_id'].size()>2)
df=df.loc[df.valid_session].drop('valid_session',axis=1)
df.nunique()

session_id     2703
timestamp     15773
item_id        3882
category          1
label             2
dtype: int64

In [32]:
# randomly sample a couple of them
#sampled_session_id=np.random.choice(df.session_id.nunique(),200,replace=False)
#df=df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id     2703
timestamp     15773
item_id        3882
category          1
label             2
dtype: int64

In [20]:
df.groupby('session_id')['item_id'].size().mean()

5.835368109507954

In [33]:
from sklearn.preprocessing import LabelEncoder

item_encoder=LabelEncoder()
df['item_id']=item_encoder.fit_transform(df.item_id)
df.head()

,session_id,timestamp,item_id,category,label
0,1,2014-04-07T10:51:09.277Z,247,0,False
1,1,2014-04-07T10:54:09.868Z,246,0,False
2,1,2014-04-07T10:54:46.998Z,248,0,False
3,1,2014-04-07T10:57:00.306Z,851,0,False
4,2,2014-04-07T13:56:37.614Z,1730,0,False


In [34]:
df['label']=df.session_id.isin(buy_df.session_id)
df.head()

,session_id,timestamp,item_id,category,label
0,1,2014-04-07T10:51:09.277Z,247,0,False
1,1,2014-04-07T10:54:09.868Z,246,0,False
2,1,2014-04-07T10:54:46.998Z,248,0,False
3,1,2014-04-07T10:57:00.306Z,851,0,False
4,2,2014-04-07T13:56:37.614Z,1730,0,False


In [35]:
df.drop_duplicates('session_id')['label'].mean()

0.097669256381798

In [23]:
df.head(5)

,session_id,timestamp,item_id,category,label
0,1,2014-04-07T10:51:09.277Z,214536502,0,False
1,1,2014-04-07T10:54:09.868Z,214536500,0,False
2,1,2014-04-07T10:54:46.998Z,214536506,0,False
3,1,2014-04-07T10:57:00.306Z,214577561,0,False
4,2,2014-04-07T13:56:37.614Z,214662742,0,False


In [38]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
import os

class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self,root,transform=None,pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root,transform,pre_transform)
        self.data,self.slices=torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['F:\CODES\codePractices\pytorch\pytorch_geometric_YooChoose\data\input\yoochoose-data\yoochoose_click_binary_1M_sess.dataset']
    
    def download(self):
        pass
    
    def process(self):
        data_list=[]
        
        #process by sessoion id
        grouped=df.groupby('session_id')
        for session_id,group in tqdm(grouped):
            session_item_id=LabelEncoder().fit_transform(group.item_id)
            group=group.reset_index(drop=True)
            group['session_item_id']=session_item_id
            node_features=group.loc[group.session_id==session_id,['session_item_id','item_id']].sort_values('session_item_id').item_id.drop_duplicates().values
                #.sort_values('session_item_id').item_id.drop_duplicates().values()
            
            ## unsqueeze(arg):表示在第arg维增加一个维度值为1的维度
            node_features=torch.LongTensor(node_features).unsqueeze(1)
            target_nodes=group.session_item_id.values[1:]
            source_nodes=group.session_item_id.values[:-1]

            edge_index=torch.tensor([source_nodes,target_nodes],dtype=torch.long)

            x=node_features
            y=torch.FloatTensor([group.label.values[0]])

            data=Data(x=x,y=y,edge_index=edge_index)
            data_list.append(data)
        # 
        data,slices=self.collate(data_list)
        torch.save((data,slices),self.processed_paths[0])      
        
dataset=YooChooseBinaryDataset(root='../data')
dataset=dataset.shuffle()
len(dataset)

train_dataset=dataset[:500]
val_dataset=dataset[500:600]
test_dataset=dataset[600:700]
len(train_dataset),len(val_dataset),len(test_dataset)
   

5

In [40]:
from torch_geometric.data import DataLoader
batch_size=10
train_loader=DataLoader(train_dataset,batch_size=batch_size)
val_loader=DataLoader(val_dataset,batch_size=batch_size)
test_loader=DataLoader(test_dataset,batch_size=batch_size)

In [41]:
num_items=df.item_id.max()+1
num_items

3882

In [46]:
import torch
from torch.nn import Sequential as Seq, Linear,ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops,add_self_loops

class SAGEConv(MessagePassing):
    def __init__(self,in_channels,out_channels):
        super(SAGEConv, self).__init__(aggr='max')
        
        self.lin=torch.nn.Linear(in_channels,out_channels)
        self.act=torch.nn.ReLU()
        self.update_lin=torch.nn.Linear(in_channels+out_channels,in_channels,bias=False)
        self.update_act=torch.nn.ReLU()
    def forward(self,x,edge_index):
        # x has the shape:[N,in_channels]
        # edge_index has shape:[2,E]
        
        edge_index,_=remove_self_loops(edge_index)
        edge_index,_=add_self_loops(edge_index,num_nodes=x.shape[0],)
        return self.propagate(edge_index,size=(x.shape[0],x.shape[0]),x=x)
    
    def message(self, x_j):
        # x_j has shape:[E,in_channels]
        
        x_j=self.lin(x_j)
        x_j=self.act(x_j)
        return x_j
    
    def update(self,aggr_out,x):
        # aggr_out has shape [N, out_channels]
        
        new_embedding=torch.cat([aggr_out,x],dim=1)
        
        new_embedding=self.update_lin(new_embedding)
        new_embedding=self.update_act(new_embedding)
        return new_embedding
      

In [89]:
embed_dim=128 # 这个维度药物SAGECon中的维度保持一致
from torch_geometric.nn import GraphConv,TopKPooling,GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap,global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1=SAGEConv(embed_dim,128)
        self.pool1=TopKPooling(128,ratio=0.8)
        self.conv2=SAGEConv(128,128)
        self.pool2=TopKPooling(128,ratio=0.8)
        self.conv3=SAGEConv(128,128)
        self.pool3=TopKPooling(128,ratio=0.8)
        
        self.item_embedding=torch.nn.Embedding(num_embeddings=df.item_id.max()+1,embedding_dim=embed_dim)
        self.lin1=torch.nn.Linear(256,128)
        self.lin2=torch.nn.Linear(128,64)
        self.lin3=torch.nn.Linear(64,1)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.act1=torch.nn.ReLU()
        self.act2=torch.nn.ReLU()
        
    def forward(self,data):
        x,edge_index,batch=data.x,data.edge_index,data.batch
        x=self.item_embedding(x)
        x=x.squeeze(1) #在维度1的位置增加一个维度
        
        x=F.relu(self.conv1(x,edge_index))
        x,edge_index,_,batch,_=self.pool1(x,edge_index,None,batch)
        x1=torch.cat([gmp(x,batch),gap(x,batch)],dim=1)
        
        x=F.relu(self.conv2(x,edge_index))
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x
        

In [52]:
device=torch.device('cpu')
model=Net().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=0.005)
crit=torch.nn.BCELoss()

NameError: name 'Net' is not defined

In [48]:
def train():
    model.train()
    
    loss_all=0
    for data in train_loader:
        data=data.to(device)
        label=data.y.to(device)
        
        optimizer.zero_grad()
        output=model(data)
        loss=crit(output,label)
        loss.backward()
        
        loss_all+=data.num_graphs*loss.item()
        optimizer.step()
    return loss_all/len(train_dataset)  

In [29]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()
    
    predictions=[]
    labels=[]
    
    with torch.no_grad():
        for data in loader:
            data=data.to(device)
            label=data.y.to(device)
            pred=model(data).detach().numpy()
            
            predictions.append(pred)
            labels.append(label)
    predictions=np.hstack(predictions) #沿着横轴拼接 列增加
    labels=np.hstack(labels)
    return roc_auc_score(labels,predictions)

In [49]:
for epoch in range(1):
    loss=train()
    train_acc=evaluate(train_loader)
    val_acc=evaluate(val_loader)
    test_acc=evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

NameError: name 'model' is not defined